In [12]:
import utm
import pickle
import pandas as pd
pd.set_option('display.max_columns', None)
# useful functions

# prints all the columns
def printcolumns(data):
    for column in data.columns:
        print (column)
        
# filters out data if a point is missing in one of the colunns
def filterblanks(columns,data,blank):
    # if blank is true, remove blanks
    # if blank is false, remove non blanks
    for c in columns:
        if blank:
            data = data[data[c].notnull()]
        else:
            data = data[data[c].isnull()]
    return data

In [27]:
#load data
data = pickle.load(open("water_data.p", "rb"))
data['LATITUDE'] = None
data['LONGITUDE'] = None
data.head()

,SHEETBAR,FLDNUM,DATE,TIME,NSAMPLES,PROJCD,LOCATCD,DATABLCK,WAVHI,WAVEHIQF,WDP,ZMAX,ZMAXQF,SECCHI,SECCHIQF,WAVCD,ZICE,ZICEQF,ZSICE,ZSICEQF,PICE,PICEQF,ZSNOW,ZSNOWQF,PSNOW,PSNOWQF,SITETYPE,STRATUM,SUMCD,SBSTRATE,VEGCOVER,VEGDENS,VEGTYPE,METHOD,UTMZONE,NORTHING,EASTING,GRIDCODE,Z15NORTH,Z15EAST,FLDEAST,FLDNORTH,DRVEAST,DRVNORTH,ACCURACY,SAMPTYPE,STAGE,RCRDCODE,LDRCODE,ROWNUM,Z,ZQF,SAMPZCD,CALCZCD,TEMP,TEMPQF,DO,DOQF,PH,PHQF,TURB,TURBQF,COND,CONDQF,VEL,VELQF,DIR,DIRQF,SMPLBAR,SNPCODE,NOXCODE,METCODE,CLCODE,SSCODE,CHSCODE,CHFCODE,PYTCODE,TNPCODE,RPTCODE,SRP,SRPQF,TSP,TSPQF,TP,TPQF,TN,TNQF,TSN,TSNQF,SS,SSQF,ORG,ORGQF,CHLS,CHLSQF,PHAEO,PHAEOQF,NHX,NHXQF,NOX,NOXQF,SI,SIQF,CA,CAQF,FE,FEQF,MN,MNQF,K,KQF,CL,CLQF,VSS,VSS_QF,CHLF,CHLFQF,CHLFI,CHLFIQF,NA,NAQF,MG,MGQF,SO4,SO4QF,HABCLASS,INSTCODE,LINKBAR,RAWNTU,RAWNTUQF,CHLcal,LATITUDE,LONGITUDE
0,41000065,1,07/26/1993,08:05,1.0,M-,9312103,NaN,NaN,NaN,2.2,2.2,NaN,40.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935508.0,538829.0,53425.9387,4935508.0,538829.0,NaN,NaN,538829.0,4935508.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,23.0,NaN,6.6,NaN,8.1,NaN,28.0,NaN,550.0,NaN,NaN,X,NaN,X,41000065.0,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.8,NaN,7.5,NaN,7.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.44875,None,None
1,41000066,1,07/26/1993,08:22,1.0,M-,9312002,NaN,NaN,NaN,8.2,8.2,NaN,42.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935908.0,538229.0,54105.9349,4935908.0,538229.0,NaN,NaN,538229.0,4935908.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,23.0,NaN,6.6,NaN,8.1,NaN,28.0,NaN,554.0,NaN,NaN,X,NaN,X,41000066.0,NaN,1,NaN,NaN,1,NaN,1,NaN,1,NaN,0.128,NaN,NaN,NaN,0.229,NaN,4.876,NaN,NaN,NaN,37.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068,NaN,2.793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7,NaN,6.0,NaN,6.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.24230,None,None
2,41000067,1,07/26/1993,08:32,1.0,M-,9312102,NaN,NaN,NaN,4.3,4.3,NaN,43.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935708.0,537829.0,53325.9389,4935708.0,537829.0,NaN,NaN,537829.0,4935708.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,22.9,NaN,6.3,NaN,8.0,NaN,24.0,NaN,564.0,NaN,NaN,X,NaN,X,41000067.0,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2,NaN,6.6,NaN,6.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.72488,None,None
3,41000068,1,07/26/1993,08:50,1.0,M-,9312003,NaN,NaN,NaN,9.1,9.1,NaN,38.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4934908.0,536429.0,53645.9347,4934908.0,536429.0,NaN,NaN,536429.0,4934908.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,22.9,NaN,6.4,NaN,8.0,NaN,28.0,NaN,563.0,NaN,NaN,X,NaN,X,41000068.0,NaN,1,NaN,NaN,1,NaN,1,NaN,1,NaN,0.139,NaN,NaN,NaN,0.212,NaN,4.257,NaN,NaN,NaN,33.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060,NaN,2.837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2,NaN,6.3,NaN,6.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.48359,None,None
4,41000069,1,07/26/1993,09:08,1.0,M-,9312104,NaN,NaN,NaN,6.7,6.7,NaN,45.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935108.0,535829.0,53585.9363,4935108.0,535829.0,NaN,NaN,535829.0,4935108.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,23.0,NaN,6.6,NaN,8.0,NaN,33.0,NaN,556.0,NaN,NaN,X,NaN,X,41000069.0,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.2,NaN,7.6,NaN,7.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.52918,None,None


In [30]:
missing = 0
for index,row in data.iterrows():
    # Base case to check if row is missing EASTING, NORTHING values; if so, print row and iterate missing_row_count, then skip the row
    if pd.isnull(row['EASTING']) or pd.isnull(row['NORTHING']):
        missing += 1
        continue
    # UTM ZONE is in Northern Hemisphere, so northern = True
    lat_long_temp = utm.to_latlon(row['EASTING'], row['NORTHING'], row['UTMZONE'], northern=True)
    # Pull values
    data.loc[index,'LATITUDE'] = lat_long_temp[0]
    data.loc[index,'LONGITUDE'] = lat_long_temp[1]
print(missing)

2


In [32]:
pickle.dump(data, open( "water_data_coords.p", "wb" ) )

In [31]:
data.head()

,SHEETBAR,FLDNUM,DATE,TIME,NSAMPLES,PROJCD,LOCATCD,DATABLCK,WAVHI,WAVEHIQF,WDP,ZMAX,ZMAXQF,SECCHI,SECCHIQF,WAVCD,ZICE,ZICEQF,ZSICE,ZSICEQF,PICE,PICEQF,ZSNOW,ZSNOWQF,PSNOW,PSNOWQF,SITETYPE,STRATUM,SUMCD,SBSTRATE,VEGCOVER,VEGDENS,VEGTYPE,METHOD,UTMZONE,NORTHING,EASTING,GRIDCODE,Z15NORTH,Z15EAST,FLDEAST,FLDNORTH,DRVEAST,DRVNORTH,ACCURACY,SAMPTYPE,STAGE,RCRDCODE,LDRCODE,ROWNUM,Z,ZQF,SAMPZCD,CALCZCD,TEMP,TEMPQF,DO,DOQF,PH,PHQF,TURB,TURBQF,COND,CONDQF,VEL,VELQF,DIR,DIRQF,SMPLBAR,SNPCODE,NOXCODE,METCODE,CLCODE,SSCODE,CHSCODE,CHFCODE,PYTCODE,TNPCODE,RPTCODE,SRP,SRPQF,TSP,TSPQF,TP,TPQF,TN,TNQF,TSN,TSNQF,SS,SSQF,ORG,ORGQF,CHLS,CHLSQF,PHAEO,PHAEOQF,NHX,NHXQF,NOX,NOXQF,SI,SIQF,CA,CAQF,FE,FEQF,MN,MNQF,K,KQF,CL,CLQF,VSS,VSS_QF,CHLF,CHLFQF,CHLFI,CHLFIQF,NA,NAQF,MG,MGQF,SO4,SO4QF,HABCLASS,INSTCODE,LINKBAR,RAWNTU,RAWNTUQF,CHLcal,LATITUDE,LONGITUDE
0,41000065,1,07/26/1993,08:05,1.0,M-,9312103,NaN,NaN,NaN,2.2,2.2,NaN,40.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935508.0,538829.0,53425.9387,4935508.0,538829.0,NaN,NaN,538829.0,4935508.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,23.0,NaN,6.6,NaN,8.1,NaN,28.0,NaN,550.0,NaN,NaN,X,NaN,X,41000065.0,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.8,NaN,7.5,NaN,7.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.44875,44.5719,-92.511
1,41000066,1,07/26/1993,08:22,1.0,M-,9312002,NaN,NaN,NaN,8.2,8.2,NaN,42.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935908.0,538229.0,54105.9349,4935908.0,538229.0,NaN,NaN,538229.0,4935908.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,23.0,NaN,6.6,NaN,8.1,NaN,28.0,NaN,554.0,NaN,NaN,X,NaN,X,41000066.0,NaN,1,NaN,NaN,1,NaN,1,NaN,1,NaN,0.128,NaN,NaN,NaN,0.229,NaN,4.876,NaN,NaN,NaN,37.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068,NaN,2.793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7,NaN,6.0,NaN,6.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.24230,44.5755,-92.5185
2,41000067,1,07/26/1993,08:32,1.0,M-,9312102,NaN,NaN,NaN,4.3,4.3,NaN,43.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935708.0,537829.0,53325.9389,4935708.0,537829.0,NaN,NaN,537829.0,4935708.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,22.9,NaN,6.3,NaN,8.0,NaN,24.0,NaN,564.0,NaN,NaN,X,NaN,X,41000067.0,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2,NaN,6.6,NaN,6.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.72488,44.5737,-92.5235
3,41000068,1,07/26/1993,08:50,1.0,M-,9312003,NaN,NaN,NaN,9.1,9.1,NaN,38.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4934908.0,536429.0,53645.9347,4934908.0,536429.0,NaN,NaN,536429.0,4934908.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,22.9,NaN,6.4,NaN,8.0,NaN,28.0,NaN,563.0,NaN,NaN,X,NaN,X,41000068.0,NaN,1,NaN,NaN,1,NaN,1,NaN,1,NaN,0.139,NaN,NaN,NaN,0.212,NaN,4.257,NaN,NaN,NaN,33.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060,NaN,2.837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2,NaN,6.3,NaN,6.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.48359,44.5666,-92.5412
4,41000069,1,07/26/1993,09:08,1.0,M-,9312104,NaN,NaN,NaN,6.7,6.7,NaN,45.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,0.0,NaN,NaN,1.0,15,4935108.0,535829.0,53585.9363,4935108.0,535829.0,NaN,NaN,535829.0,4935108.0,NaN,NaN,NaN,111.0,111.0,1.0,0.2,NaN,NaN,SF,23.0,NaN,6.6,NaN,8.0,NaN,33.0,NaN,556.0,NaN,NaN,X,NaN,X,41000069.0,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.2,NaN,7.6,NaN,7.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.52918,44.5684,-92.5488


In [3]:
# filter
print("Before filtering: ",data.shape)
#printcolumns(data)
data.drop(data.columns.difference(['TN','TP','TPQF','TNQF','SS','SSQF',
                                         'TURB','TURBQF','WDP',
                                         'TEMP','TEMPQF','DO','DOQF','COND',
                                         'CONDQF','VEL','VELQF','FLDEAST',
                                         'FLDNORTH','PROJCD','FLDNUM','DATE',
                                  'LOCATCD','STRATUM','CHLcal','SECCHI','SECCHIQF']), 1, inplace=True)
print("After filtering columns: ",data.shape)
print("Now filtering sampling design")
data = data[(data.PROJCD == "M-")]
print("After filtering sampling design: ",data.shape)
print("Now filtering Pool 13")
data = data[(data.FLDNUM == 3)]
print("After filtering Pool 13: ",data.shape)
print("Now adding a year column")
data["YEAR"] = pd.DatetimeIndex(data["DATE"]).year
print(data.shape)
print("Adding a timecode column")
data["TIME CODE"] = data["LOCATCD"].astype(str).apply(lambda x: x[3])
print(data.shape)
#print("Now filtering summer by timecode 2 (June Through August)")
#data = data[data["TIME CODE"] == '2']
#print(data.shape)
print("Now filtering by backwater lakes")
data = data[data.STRATUM == 3]
print(data.shape,data["YEAR"].unique())
print("Filtering out bad data")
QFcols = ['TPQF','TNQF','SSQF','TURBQF','TEMPQF','DOQF','CONDQF','VELQF','SECCHIQF']
data = filterblanks(QFcols,data,False)
print(data.shape,data["YEAR"].unique())
print("Dropping all blank columns")
data.drop(['PROJCD','FLDEAST','FLDNORTH','TPQF','TNQF','SSQF','TURBQF','TEMPQF','DOQF',
                                         'CONDQF','VELQF','SECCHIQF'], 1, inplace=True)
print(data.shape)
print("Filtering out points with blank entries in at least one of the columns")
cols = ['TN','TP','TEMP','DO','TURB','COND','VEL','SS','WDP','CHLcal','SECCHI']
#for col in cols:
#    print(col,data[col].isna().sum())
data = filterblanks(cols,data,True)
print(data.shape,data["YEAR"].unique())
#print("Filtering out all years but 1993, 1994, 1995, 1996")
#data = data[(data['YEAR']==1993) | (data['YEAR']==1994) | (data['YEAR']==1995) | (data['YEAR']==1996)]
#print(data.shape)

Before filtering:  (106052, 131)
After filtering columns:  (106052, 27)
Now filtering sampling design
After filtering sampling design:  (106052, 27)
Now filtering Pool 13
After filtering Pool 13:  (17991, 27)
Now adding a year column
(17991, 28)
Adding a timecode column
(17991, 29)
Now filtering by backwater lakes
(8097, 29) [1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2004 2005 2006 2007
 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Filtering out bad data
(813, 29) [1993 1994 1995 1996 1998 2000 2001 2002 2004 2005 2006 2007 2008 2009
 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Dropping all blank columns
(813, 17)
Filtering out points with blank entries in at least one of the columns
(258, 17) [1993 1994 1995 1996]


In [5]:
data.index

Int64Index([39655, 39661, 39666, 39687, 39690, 39691, 39692, 39694, 39695,
            39696,
            ...
            41613, 41615, 41616, 41624, 41629, 41639, 41654, 41657, 41658,
            41661],
           dtype='int64', length=258)

In [59]:
data.isna().sum()

FLDNUM       0
DATE         0
LOCATCD      0
WDP          0
SECCHI       0
STRATUM      0
TEMP         0
DO           0
TURB         0
COND         0
VEL          0
TP           0
TN           0
SS           0
CHLcal       0
YEAR         0
TIME CODE    0
dtype: int64

None
